# CONVERT FILES WITH DUCK_DB

In [1]:
import pandas as pd
import duckdb
import os
import time
import numpy as np

In [2]:
ZIP_PATH = r"..\data_processed\\guo_subs_europee\\"
ZIP_FOLDER = "guos_eu"

In [3]:
def time_it(func):
    """Decorator to measure the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs) 
        end_time = time.time() 
        elapsed_time = end_time - start_time  
        print(f"Execution time: {elapsed_time:.4f} seconds")  
        return result
    return wrapper

## Ownership history

In [4]:
eu27_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", 
    "DK", "EE", "FI", "FR", "DE", "GR", 
    "HU", "IE", "IT", "LV", "LT", "LU", 
    "MT", "NL", "PL", "PT", "RO", "SK", 
    "SI", "ES", "SE"
]

In [5]:

TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed\\*.parquet" 
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\firmographics_historicals_processed"


def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"\\firmographics_historicals_{country}.parquet"
            
            
    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            {year} AS year,
            key_financials.number_of_employees AS guo_number_of_employees,
            key_financials.closing_date AS guo_closing_date
        FROM 
            '{path}' AS main
        LEFT JOIN 
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number
        LEFT JOIN
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_guo
        ON
            main.guo_25 = firmographics_guo.bvd_id_number
        LEFT JOIN 
            (SELECT * 
             FROM '{COUNTRY_KEY_FINANCIALS}'
             WHERE 
                year = {year}
            ) AS key_financials
        ON
            main.subsidiary_bvd_id = key_financials.bvd_id_number   -- CHANGE HERE FOR SUBS
        AND
            key_financials.year = {year}

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."subsidiary_bvd_id" LIKE '{country}%'
    """
    

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df



In [6]:
@time_it
def fetch_and_convert_to_excel(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    os.makedirs(output_path, exist_ok=True)

    # Split the DataFrame into 5 chunks
    chunks = 5
    indices = np.array_split(df.index, chunks)
    slices = [df.iloc[idx] for idx in indices]

    for i, s in enumerate(slices):
        s.to_csv(f"{output_path}\\{country}_{year}_chunk_{i+1}.csv", index=False)
    
        # s.to_stata(f"{output_path}\\{country}_{year}_chunk_{i+1}.dta", write_index=False)
        print(f"Data for {country} - {year} - slice_{i+1} has been converted.")
        

### Test

In [7]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
country = "DE"
year = "2009"
path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)
# fetch_and_convert_to_excel(country=country, year=year, path=path, output_path=output_path)

DE - 2009...


In [9]:
import time
import json

errors = {}
OUTPUT_PATH = r"..\data_processed\\guo_subs_europee\\subs_eu\\" 

total_start = time.time()
start_time = time.time()

eu27_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", 
    "DK", "EE", "FI", "FR", "DE",
    "GR", "HU", "IE", 
    "IT", 
    "LV", "LT", "LU", 
    "MT", "NL", "PL", "PT", "RO", "SK", 
    "SI", "ES", "SE"
]

for country in eu27_countries:
    for year in range(2022, 2023):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)  
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

with open("errors_guos.json", "w") as f:
    json.dump(errors, f)


AT - 2007...
Data for AT - 2007 - slice_1 has been converted.
Data for AT - 2007 - slice_2 has been converted.
Data for AT - 2007 - slice_3 has been converted.
Data for AT - 2007 - slice_4 has been converted.
Data for AT - 2007 - slice_5 has been converted.
Execution time: 24.0440 seconds
Data for AT - 2007 has been converted to Stata in 24.04 seconds.
AT - 2008...
Data for AT - 2008 - slice_1 has been converted.
Data for AT - 2008 - slice_2 has been converted.
Data for AT - 2008 - slice_3 has been converted.
Data for AT - 2008 - slice_4 has been converted.
Data for AT - 2008 - slice_5 has been converted.
Execution time: 23.2387 seconds
Data for AT - 2008 has been converted to Stata in 47.28 seconds.
AT - 2009...
Data for AT - 2009 - slice_1 has been converted.
Data for AT - 2009 - slice_2 has been converted.
Data for AT - 2009 - slice_3 has been converted.
Data for AT - 2009 - slice_4 has been converted.
Data for AT - 2009 - slice_5 has been converted.
Execution time: 23.4727 seconds


In [10]:
# open errors
import json 

with open("errors_guos.json", "r") as f:
    errors = json.load(f)

errors

{}

In [11]:
# import shutil

# ZIP_PATH = r"..\data_processed\\guo_subs_europee\\subs_eu"
# ZIP_FOLDER = r"..\data_processed\\guo_subs_europee"

# shutil.make_archive(
#     base_name=fr"{ZIP_FOLDER}\subs_eu_zipped",  
#     format="zip",
#     base_dir=ZIP_PATH  
# )

'..\\data_processed\\\\guo_subs_europee\\subs_eu_zipped.zip'